In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import os
import pandas as pd
import seaborn as sns
import fiona
import sklearn
import xgboost as xgb

In [4]:
#Upload the selection of glaccier i will use in the model
selection = []

with open('/Users/francesco/Desktop/Thesis/Data/dataset_filtered' + '/selecao.txt', 'r') as f:
    for line in f:
        selection.append(int(line.strip()))

common_years = np.arange(2005,2017,1)

print('The shape of the selection is: ', np.shape(selection))

The shape of the selection is:  (128,)


In [5]:
path_to_tif = '/Users/francesco/Desktop/Data/GEOTIFFs/'
shapefile_path = '/Users/francesco/Desktop//Thesis/Data/ice_shelf.shp'

ids = []
Names = []
regions = []
areas = []
lats = []
lons = []

shapefile=fiona.open(shapefile_path)
print(shapefile.schema['properties'])

for feature in shapefile:

    id = feature['properties']['id']
    name=feature['properties']['name']
    area=feature['properties']['area_fra']
    lat = feature['properties']['latitude']
    lon = feature['properties']['longitude']
    geometry=feature['geometry']
    if geometry is None:
        continue
    region = feature['properties']['regions']

    #Taking the info
    ids.append(id)
    Names.append(name)
    regions.append(region)
    areas.append(area)
    lats.append(lat)
    lons.append(lon)

#Creating the dataframe
df_region = pd.DataFrame(index=ids)
df_region ['id'] = ids
df_region ['Name'] = Names
df_region ['Region'] = regions
df_region ['Area'] = areas
df_region ['Latitude'] = lats
df_region ['Longitude'] = lons

#Selecting the region and sorting the dataframe
df_region = df_region[df_region['id'].isin(selection)]
df_region = df_region.sort_values(by=['id'])
#drop id column
df_region = df_region.drop(['id'], axis=1)

{'id': 'int:10', 'name': 'str:30', 'regions': 'str:11', 'type': 'str:2', 'is_index': 'int:10', 'ice_shelf_': 'str:50', 'measures_n': 'str:50', 'latitude': 'float:24.15', 'longitude': 'float:24.15', 'thickness_': 'float:24.15', 'thicknes_1': 'float:24.15', 'smb_mean_9': 'float:24.15', 'smb_mean_1': 'float:24.15', 'ss_gl_flux': 'float:24.15', 'ss_gl_fl_1': 'float:24.15', 'ss_calving': 'float:24.15', 'ss_calvi_1': 'float:24.15', 'model_gl_f': 'float:24.15', 'instant_co': 'float:24.15', 'mass_loss_': 'float:24.15', 'mass_los_1': 'float:24.15', 'mass_los_2': 'float:24.15', 'mass_los_3': 'float:24.15', 'mass_los_4': 'float:24.15', 'mass_los_5': 'float:24.15', 'mass_los_6': 'float:24.15', 'mass_los_7': 'float:24.15', 'mass_los_8': 'float:24.15', 'mass_los_9': 'float:24.15', 'mass_los10': 'float:24.15', 'mass_los11': 'float:24.15', 'mass_los12': 'float:24.15', 'mass_los13': 'float:24.15', 'mass_los14': 'float:24.15', 'mass_los15': 'float:24.15', 'mass_los16': 'float:24.15', 'mass_los17': 'floa

In [7]:
#Loading the dataset
dataset_directory = '/Users/francesco/Desktop/Thesis/Data/dataset_filtered/'

# Basal Melting
bm = pd.read_csv(dataset_directory + '/bm.csv', index_col=0)
bm = bm.sort_values(by=['id']) #sorting the glaciers by their index
bm = bm.loc[bm.index.isin(selection)] #selecting the glaciers, according to their index
bm = bm[common_years.astype(str)] #selecting the common years
bm = bm.sort_index() #sorting the glaciers by their index

#Load the calving data
calving = pd.read_csv(dataset_directory+ '/df_calving_from_shp_negative_and_positive.csv', index_col=0)
calving = calving.loc[calving.index.isin(selection)]
calving = calving[common_years.astype(str)]
#sort the calving by its index
calving = calving.sort_index()

#Load the ice concentration data
i_c = pd.read_csv(dataset_directory + '/ice_c_avg_extended_front.csv', index_col=0)
i_c = i_c.loc[i_c.index.isin(selection)]
i_c = i_c[common_years.astype(str)]
i_c = i_c.sort_index()

#Load the ice velocity data
i_v = pd.read_csv(dataset_directory + '/velocity_80_percentile_extended_front_2011_2012_linear_trend.csv', index_col=0)
i_v = i_v.loc[i_v.index.isin(selection)]
i_v = i_v[common_years.astype(str)]
i_v = i_v.sort_index()

#Load the ice thickness data
i_t = pd.read_csv(dataset_directory + '/thickness_avg_extended front.csv', index_col=0)
i_t = i_t.loc[i_t.index.isin(selection)]
i_t = i_t[common_years.astype(str)]
i_t = i_t.sort_index()

index = bm.index

In [8]:
#make a dataset with all the variables
dataset = pd.concat([bm, calving, i_c, i_v, i_t], axis=1, keys=['bm', 'calving', 'i_c', 'i_v', 'i_t'])
dataset.columns.names = ['Variables', 'Years']
dataset.index.names = ['Glaciers']

#i want to split the dataset in two, one with the variables and one with the target. The target is ice calving
dataset_target = dataset['calving']
dataset_variables = dataset.drop(['calving'], axis=1)

In [9]:
#Folders division

folder_1 = [2,4,6,7,8,15,18,16,34,45,135,163,95]
folder_2 = [3,29,10,12,31,30,19,24,35,50,86,104,139]
folder_3 = [65,43,37,17,41,32,38,40,36,52,143,146,125]
folder_4 = [69,67,63,26,46,33,64,44,70,53,123,145,170]
folder_5 = [75,78,89,39,47,81,77,54,71,55,118,101,117]
folder_6 = [76,80,114,58,48,84,83,66,90,56,140,144,164]
folder_7 = [88,82,120,61,57,92,85,105,100,73,158,124,169]
folder_8 = [96,107,121,62,60,127,87,108,102,91,161,122,160]

folder_9 = [129,112,156,68,97,132,93,109,115,98,148,166] #test
folder_10 = [131,147,157,72,99,136,116,111,119,110,162,159] #test

#merge the folders, in order to have a single list
from itertools import chain


train = [folder_1, folder_2, folder_3, folder_4, folder_5, folder_6, folder_7, folder_8]
test = [folder_9, folder_10]


In [11]:
#Operate block division
cv_block_1 = dataset.loc[folder_1]
cv_block_2 = dataset.loc[folder_2]
cv_block_3 = dataset.loc[folder_3]
cv_block_4 = dataset.loc[folder_4]
cv_block_5 = dataset.loc[folder_5]
cv_block_6 = dataset.loc[folder_6]
cv_block_7 = dataset.loc[folder_7]
cv_block_8 = dataset.loc[folder_8]

cv_blocks = [cv_block_1, cv_block_2, cv_block_3, cv_block_4, cv_block_5, cv_block_6, cv_block_7, cv_block_8]

#Selecting the variables
cv_block_1_variables = cv_block_1.drop(['calving'], axis=1)
cv_block_2_variables = cv_block_2.drop(['calving'], axis=1)
cv_block_3_variables = cv_block_3.drop(['calving'], axis=1)
cv_block_4_variables = cv_block_4.drop(['calving'], axis=1)
cv_block_5_variables = cv_block_5.drop(['calving'], axis=1)
cv_block_6_variables = cv_block_6.drop(['calving'], axis=1)
cv_block_7_variables = cv_block_7.drop(['calving'], axis=1)
cv_block_8_variables = cv_block_8.drop(['calving'], axis=1)

cv_block_variables = [cv_block_1_variables, cv_block_2_variables, cv_block_3_variables, cv_block_4_variables, cv_block_5_variables, cv_block_6_variables, cv_block_7_variables, cv_block_8_variables]

#Selecting the target
cv_block_1_target = cv_block_1['calving']
cv_block_2_target = cv_block_2['calving']
cv_block_3_target = cv_block_3['calving']
cv_block_4_target = cv_block_4['calving']
cv_block_5_target = cv_block_5['calving']
cv_block_6_target = cv_block_6['calving']
cv_block_7_target = cv_block_7['calving']
cv_block_8_target = cv_block_8['calving']

cv_block_targets = [cv_block_1_target, cv_block_2_target, cv_block_3_target, cv_block_4_target, cv_block_5_target, cv_block_6_target, cv_block_7_target, cv_block_8_target]


In [ ]:
#Trying to immplement the cross validation in blocks

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


#rf = RandomForestRegressor(n_estimators=1000, random_state=42)
best_parameters = []


for i in range(len(cv_blocks)):
    print(f"Fold {i +1 }:")

    #Extracting the training an testing blocks from the list
    test_block = cv_blocks[i]
    train_blocks = cv_blocks[:i] + cv_blocks[i+1:]

    #Extracting the training and testing variables and targets
    X_test = test_block.drop(['calving'], axis=1)
    y_test = test_block['calving']

    X_train = pd.concat(train_blocks).drop(['calving'], axis=1)
    y_train = pd.concat(train_blocks)['calving']

    # Fit the model on the training data
    xgb_model_cv = GridSearchCV(xgb.XGBRegressor(), grid)

    # Make predictions on the test data
    xgb_model_cv.fit(X_train, y_train)
    y_pred = xgb_model_cv.predict(X_test)

    '''
    plt.scatter(y_test, y_pred)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')

    #plot the line y= x (diagonal of the first and third quadrant)
    max_value = np.max([y_test, y_pred])
    min_value = np.min([y_test, y_pred])
    x = np.linspace(min_value, max_value, 100)
    plt.plot(x, x, color='black')
    plt.title('Fold ' + str(i+1))


    plt.show()
    '''

    # Evaluate the model's performance
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)

    best_parameters.append(rf_cv.best_params_)

In [17]:
#perform the cross validation with xgboost
import xgboost as xgb

#Create the parameters for the model
params = {
    'max_depth': 3,
    'learning_rate': 0.1,
    'objective': 'reg:squarederror',
    'n_estimators': 1000
}

#Create the model
model = xgb.XGBRegressor(**params)

#Create the cross validation

from sklearn.model_selection import cross_val_score

#Create the cross validation
scores = []
for i in range(len(cv_blocks)):
    score = cross_val_score(model, cv_block_variables[i], cv_block_targets[i], scoring='neg_mean_squared_error')
    scores.append(score)
    print('The score for the block ', i, ' is: ', score)



The score for the block  0  is:  [  -10.64023056 -2756.96996508 -6791.06483503 -4248.16741749
   -40.67776316]
The score for the block  1  is:  [-2097.97989868    -3.72819618 -2172.15697605 -3006.25288465
   -32.6781086 ]
The score for the block  2  is:  [-73.82074988  -5.85317016 -96.42872844 -55.05891223 -38.01497757]
The score for the block  3  is:  [-1.32432348e+01 -8.06455971e+01 -4.07830935e+03 -2.08674670e+02
 -2.03173624e+00]
The score for the block  4  is:  [-6.34946085e+00 -5.67810557e+00 -6.70933302e+03 -1.01090205e+04
 -2.05889142e+01]
The score for the block  5  is:  [   -67.40371266 -26726.20334862   -173.01548106 -40676.53896089
   -115.63300012]
The score for the block  6  is:  [ -30.13754018 -337.78433294 -350.12452017  -99.08480173  -57.6123823 ]
The score for the block  7  is:  [ -8674.12965997  -8487.02351694   -346.33354902 -12407.00786592
   -383.13696689]
